**Desarrollado por:** Javier Fernando Botía Valderrama

*Docente del Departamento de Ingeniería de Sistemas*

**Materia:** Análisis Avanzado de Datos

**Departamento:** Ingeniería Aeroespacial

**Facultad de Ingeniería - Universidad de Antioquia**

# Tipo de Datos - Parte 1

In [ ]:
import pandas as pd
import numpy as np
import io
import requests
from sklearn.preprocessing import OneHotEncoder # Librería donde está el método One-Hot Encoder
from sklearn.preprocessing import LabelEncoder # Codificación de etiquetas
from collections import defaultdict # Para generar un diccionario nuevo de valores
from sklearn.preprocessing import MultiLabelBinarizer # Codificación binarizador Múltiple

## Definción de matriz de datos

**Datos**: Son observaciones de fenómenos del mundo real o de sucesos. Cada parte de los datos representa un aspecto limitado de la realidad pero una colección de ellos muestra una "imagen o representación" global de varios acontencimientos del mundo real. 

**Definición formal de los datos:** Dado un conjunto de datos, $X$, decimos que $x_{n,d}$ es un elemento de $X$ tal que $n$ es una observación o *muestra* y $d$ es un descriptor o *característica*, donde $X$ es una matriz representado como:

$$X = \begin{bmatrix} x_{1,1} & \cdots& x_{1,d} & \cdots&  x_{1,D} \\
\vdots & \cdots& \ddots & \cdots&  \vdots \\
x_{n,1} & \cdots& x_{n,d} & \cdots&  x_{n,D} \\
\vdots & \cdots& \ddots & \cdots&  \vdots \\
x_{N,1} & \cdots& x_{N,d} & \cdots&  x_{N,D} \\
\end{bmatrix}_{N x D} $$

siendo $N$ el número de muestras u observaciones y $D$ el número de características.

**Característica o Descriptor o Atributo:** Es una representación numérica de un conjunto de datos qe puede representar una cantidad de acontecimientos, sucesos o hechos del mundo real. Por lo general, las características se difieren desde el tipo de dato (que se explicará más adelante) que este disponible en el conjunto de datos. Una característica puede representar, por ejemplo, temperatura, presión, número de clientes por día, fechas, etc.

Los datos pueden ser de diferentes fuentes: **categóricas, cuantitativas, cualitativas, mixtas, imágenes, texto, etc.**. Para esta sección, se mostrará las dos primeras.

## Analizando datos categóricos

**Datos Categóricos:** Son datos que representa categorías o etiquetas, por ejemplo, las ciudades más grandes de un país, las cuatro temporadas del año, los equipos de fútbol en España, etc. Por lo general, los datos categóricos son *finitos* y los valores categóricos se pueden representar numéricamente. No obstante, se puede presentar dos escenarios: 

*Características Ordinales:* Son valores categóricos que pueden ser clasificados de forma ordenada, por ejemplo, que $0 \leq 1 \leq 10$.

*Características Nominales:* Son valores categóricos que no tienen ningún orden. Por ejemplo, Real Madrid es más grande o más pequeño que Barcelona ¿Cómo saber que es más grande y más pequeño?

Considerando ambos escenarios, dentro de los datos categóricos, existen varios métodos para codificar las variables categóricas que analizaremos a continuación:

In [ ]:
url = "https://raw.githubusercontent.com/javierfernandobotia/AnalisisAvanzadoDatos/main/adult.data"
download = requests.get(url).content
columnas = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'income']
data = pd.read_table(io.StringIO(download.decode('utf-8')), sep=',', 
                     names = columnas, decimal = ',', header = None) 
# Esimportante codificarlo a utf-8
display(data.head())

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   gender          32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


### One-Hot Encoding

**Codificación de un solo paso (One-Hot Encoding):** Es un método que codifica en un grupo de bits los valores categóricos. Cada bit representa una posible categórica y por regla general, la variable categórica no puede pertenecer a múltiples categorías de una sola vez, por lo que solamente 1 bit en el grupo puede estar activo. Un ejemplo sencillo es el siguiente:

Equipo de futbol $\,\,\,\,\,$ e1 $\,\,\,\,\,$ e2 $\,\,\,\,\,$ e3 $\,\,\,\,\,$ e4

Real Madrid$\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,$ 1 $\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,$ 0

Barcelona$\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 1 $\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,$ 0

Sevilla $\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 1 $\,\,\,\,\,\,\,\,$ 0

Alaves $\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,$ 1

In [ ]:
data['workclass'].astype('category')

0                State-gov
1         Self-emp-not-inc
2                  Private
3                  Private
4                  Private
               ...        
32556              Private
32557              Private
32558              Private
32559              Private
32560         Self-emp-inc
Name: workclass, Length: 32561, dtype: category
Categories (9, object): [' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ...,
                         ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay']

In [ ]:
data['education'].astype('category')

0          Bachelors
1          Bachelors
2            HS-grad
3               11th
4          Bachelors
            ...     
32556     Assoc-acdm
32557        HS-grad
32558        HS-grad
32559        HS-grad
32560        HS-grad
Name: education, Length: 32561, dtype: category
Categories (16, object): [' 10th', ' 11th', ' 12th', ' 1st-4th', ..., ' Masters', ' Preschool',
                          ' Prof-school', ' Some-college']

In [ ]:
data['marital-status'].astype('category')

0              Never-married
1         Married-civ-spouse
2                   Divorced
3         Married-civ-spouse
4         Married-civ-spouse
                ...         
32556     Married-civ-spouse
32557     Married-civ-spouse
32558                Widowed
32559          Never-married
32560     Married-civ-spouse
Name: marital-status, Length: 32561, dtype: category
Categories (7, object): [' Divorced', ' Married-AF-spouse', ' Married-civ-spouse',
                         ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed']

In [ ]:
data['relationship'].astype('category')

0         Not-in-family
1               Husband
2         Not-in-family
3               Husband
4                  Wife
              ...      
32556              Wife
32557           Husband
32558         Unmarried
32559         Own-child
32560              Wife
Name: relationship, Length: 32561, dtype: category
Categories (6, object): [' Husband', ' Not-in-family', ' Other-relative', ' Own-child',
                         ' Unmarried', ' Wife']

In [ ]:
data['race'].astype('category')

0         White
1         White
2         White
3         Black
4         Black
          ...  
32556     White
32557     White
32558     White
32559     White
32560     White
Name: race, Length: 32561, dtype: category
Categories (5, object): [' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White']

In [ ]:
data['gender'].astype('category')

0           Male
1           Male
2           Male
3           Male
4         Female
          ...   
32556     Female
32557       Male
32558     Female
32559       Male
32560     Female
Name: gender, Length: 32561, dtype: category
Categories (2, object): [' Female', ' Male']

In [ ]:
data['native-country'].astype('category')

0         United-States
1         United-States
2         United-States
3         United-States
4                  Cuba
              ...      
32556     United-States
32557     United-States
32558     United-States
32559     United-States
32560     United-States
Name: native-country, Length: 32561, dtype: category
Categories (42, object): [' ?', ' Cambodia', ' Canada', ' China', ..., ' Trinadad&Tobago',
                          ' United-States', ' Vietnam', ' Yugoslavia']

In [ ]:
datos_categoricos = data[['workclass','education', 'marital-status', 'occupation', 'relationship', 
                          'race', 'gender', 'native-country']]# Seleccionamos aquellas características con datos
datos_categoricos.shape  # Tamaño de los datos categóricos                                                                                                                                      # categóricos

(32561, 8)

In [ ]:
ENC = OneHotEncoder(handle_unknown='ignore') # One-Hot Encoding. El parámetro handle_unknown define si una característica categórica está presente durante la transformación
                                             # Si se utiliza la opción 'ignore' significa que si hay una categoría desconocida y es encontrado durante la transformación,
                                             # el resultado de las columnas codificadas serán 0. Si se utiliza 'error', entonces se codificara las categorías desconocidas como None.
ENC.fit(datos_categoricos) # Aplicamos el método One-Hot Enconding

OneHotEncoder(handle_unknown='ignore')

In [ ]:
ENC.categories_ # Determina las categorias cada característica determinadas durante la transformación.

[array([' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private',
        ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
       dtype=object),
 array([' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th',
        ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate',
        ' HS-grad', ' Masters', ' Preschool', ' Prof-school',
        ' Some-college'], dtype=object),
 array([' Divorced', ' Married-AF-spouse', ' Married-civ-spouse',
        ' Married-spouse-absent', ' Never-married', ' Separated',
        ' Widowed'], dtype=object),
 array([' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair',
        ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners',
        ' Machine-op-inspct', ' Other-service', ' Priv-house-serv',
        ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support',
        ' Transport-moving'], dtype=object),
 array([' Husband', ' Not-in-family', ' Other-relative', ' Own-child',
        ' Unmarried'

Como hay datos faltantes simbolizados como ?, vamos a reemplazar por la palabra *Nothing*, para lograr más adelante la codificación de los datos categóricos

In [ ]:
datos_categoricos = datos_categoricos.replace(' ?','Nothing') # Nota: se agrego un espacio a la categoria ? 
                                                              # porque la base de datos se observa ese espacio

Una vez realizado este paso, vamos a proceder a aplicar de nuevo el método One-Hot Encoding

In [ ]:
ENC.fit(datos_categoricos) # Aplicamos el método One-Hot Enconding

Una vez verificado el cambio de la categoria ? por *Nothing*, vamos a transformar los datos

In [ ]:
data_ENC = ENC.transform(datos_categoricos).toarray() # Codificación One-Hot Encoding
display(data_ENC)

In [ ]:
print('Número de características codificadas: ', data_ENC.shape[1])

In [ ]:
data_ENC.shape

Debido a la forma como se codifica los datos categóricos, el número de características aumenta debido a que el método considera cada columna de la base de datos con la nomenclatura: Caracteristica/Categoría. Por ejemplo: workclass/ Federal-gov, workclass/ Local-gov, etc... A continuación, se puede aplicar la transformada inversa del método One-Hot Encoding

In [ ]:
ENC.inverse_transform(data_ENC) # Decodificación

Con la transformada inversa, se verifica si la codificación ejecutada coincide con las categorías originales de la base de datos

Ahora vamos a organizar nuestra base de datos con los nombres caracteristica/categoria para cada columna de la base de datos categóricos

In [ ]:
Car_Cat = ENC.get_feature_names(['workclass','education', 'marital,status', 'occupation', 
                                 'relationship', 'race', 'gender', 'native-country'])

In [ ]:
data_ENC = pd.DataFrame(data_ENC, columns = Car_Cat)
display(data_ENC)

En la codificación de un solo paso, utiliza más de un bit para su codificación, lo cual el número de características codificadas aumenta. Por ejemplo, si hay k-1 de los bits son 0, entonces el último bit debe ser 1 debido a que la variable debe tomar un 1 en k valores. Lo anterior se debe a que la suma de todos los bits por cada muestra u observación es igual a 1:

$n=1\,\,\,\, e_1 + e_2 + e_k = 1$

$n=2\,\,\,\, e_1 + e_2 + e_k = 1$

$\vdots \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\, \vdots$

$n=N\,\,\,\, e_1 + e_2 + e_k = 1$

donde cada $e_c$ es 0 o 1.

**Limitante de la codificación de una sola vez:** Genera demasiados grados de libertad o k valores que incrementa la dimensionalidad de los datos cuando las variables solo requiere k-1 valores.

### Dummy Encoding

**Codificación simulada (Dummy Coding)**: Este método se encarga de remover los grados de libertad extra genera la codificación de una sola vez o one-hot encoding, usando solamente *k-1 características*. En este caso, una característica es puesta debajo de las otras características por medio de un vector de todos 0, lo cual es la **categoría de referencia**. Lo anterior se puede representar mediante el siguiente ejemplo:

Equipo de futbol $\,\,\,\,\,$ e1 $\,\,\,\,\,$ e2 $\,\,\,\,\,$ e3

Real Madrid$\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,$ 1 $\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 0 

Barcelona$\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 1 $\,\,\,\,\,\,\,\,\,$ 0 

Sevilla $\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 1 

Alaves $\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 0 $\,\,\,\,\,\,\,\,\,$ 0

In [ ]:
data_cat_Dummy = pd.get_dummies(datos_categoricos, drop_first = True) # Codificación simulada. El parámetro drop_first permite sacar las k-1 simulaciones 
                                                         # de los niveles categóricos, eliminando el primer nivel de codificación.
display(data_cat_Dummy)

Observen que la codificación Dummy Encoding reduce una parte del número de características codificadas con respecto al método de One-Hot Encoding. Además, en pocas líneas de código se logra hacer los mismos pasos con respecto al método One-Hot Encoding

### Codificación de Efectos

Es un método similar a la codificaciíon simulada o Dummy Encoding pero la categoría de referencia es representado por vectores de todos "-1". Este tipo de codificación es muy usual cuando se desea crear modelos de regresión lineal a partir de datos categóricos (que lo verán en detalle en la materia de aprendizaje automático 1).

In [ ]:
data_cat_Dummy_Efecto = data_cat_Dummy.copy()
data_cat_Dummy_Efecto = data_cat_Dummy.replace(0,'-1').astype('float32') # Reemplazar 0 por -1
display(data_cat_Dummy_Efecto)

### Codificación de Etiquetas

**Codificación de etiquetas (LabelEncoder):** Es un método de codificación donde realiza una etiquetación de los valores categóricos de cada características desde 0 hasta $N_{clases}-1$. Por ejemplo, si en una característica hay un vector de valores categóricos donde solo representa "Real Madrid", "Barcelona" y "Sevilla", entonces "Real Madrid" es etiquetado como 0 y "Barcelona" es etiquetado como 1 y "Sevilla" es etiquetado como 2.

In [ ]:
LE = defaultdict(LabelEncoder)
data_LE = datos_categoricos.apply(lambda x: LE[x.name].fit_transform(x))
display(data_LE)

In [ ]:
data_LE.info()

In [ ]:
Datos_Completos_con_LE = np.vstack((data['age'], data_LE['workclass'], data['fnlwgt'], data_LE['education'], data['education-num'],
                                    data_LE['marital-status'],data_LE['occupation'], data_LE['relationship'], data_LE['race'], data_LE['gender'], data['capital-gain'],
                                    data['capital-loss'],data['hours-per-week'],data_LE['native-country'], data['income'] )).T # Impotante aplicar la transpuesta

Datos_Completos_con_LE = pd.DataFrame(Datos_Completos_con_LE, columns = columnas)

display(Datos_Completos_con_LE)

Falta transformar la característica income

In [ ]:
LE_New = LabelEncoder()
Datos_Completos_con_LE['income'] = LE_New.fit_transform(Datos_Completos_con_LE['income'])
display(Datos_Completos_con_LE)

Vamos a inspeccionar si aún presenta algún dato faltante

In [ ]:
Datos_Completos_con_LE = Datos_Completos_con_LE.replace(" ?", np.nan) # Reemplazamos los datos ? por datos NaN o nulos si los hay
Datos_Completos_con_LE.info()

Ahora tranformamos el tipo de variable `object` en entero.


In [ ]:
Data_Final = []

for d in range(Datos_Completos_con_LE.shape[1]):
  Data_Final.append(pd.to_numeric(Datos_Completos_con_LE.iloc[:,d], errors='coerce'))

Data_Final = pd.DataFrame(np.array(Data_Final).T, columns = columnas)
display(Data_Final)

In [ ]:
Data_Final.info()

### Codificación Binarizador

**Codificación binarizador (LabelBinarizer):** Es un método de codificación llamada también *one-vs-all (OvA)*, en cual amplia un clasificador binario a problemas de múltiples etiquetas.

In [ ]:
MLB = MultiLabelBinarizer()
Datos_LB = MLB.fit_transform([(1, 2), (3, 2), (4, 2), (5, 3)]) 
display(Datos_LB)

In [ ]:
MLB.classes_

### Ventajas y Desventajas de la Codificación de Valores Categóricos

**Ventajas de la codificación de las valores categóricos:** Por lo general, la codificación de una sola vez o one-hot encoding genera redudancia de los datos que es útil para validar modelos de machine learning de un mismo problema que se esta resolviendo con la base de datos. Por otra parte, la codificación simulada o dummy coding no genera redudancia de los datos y mejora la interpretabilidad de los modelos de machine learning. En el caso de la codificación de efectos o effect coding, tampoco genera redundancia de los datos con el simple hecho que se utiliza -1.0 para la categoría de referencia.

**Desventajas de la codificación de los valores categóricos:** Para el caso de one-hot encoding, la redundancia de los datos genera problemas de interpretabilidad de los modelos cuando el tamaño original de los datos tiene una alta dimensionalidad, lo cual hace díficil su aplicabilidad en esos escenarios. Por otra parte, dummy coding no es fácil de utilizar cuando existe un gran número de datos faltantes porque los asume como categorías de referencia, generando redundancia de vectores de 0. En cambio, effect coding evita el problema de dummy coding con los datos faltantes a relacionarlos como vectores de -1.0, pero almacenar estas cantidades de -1.0 genera un alto costo de máquina y almacenamiento computacional.

## Analizando datos cuantitativos

### Aplicación por columnas y funciones múltiples

Son datos que expresan cantidades numéricas expresados como números naturales, enteros, racionales o reales. Por lo general, los datos cuantitativos expresan valores tomados de un conjunto de sensores, de recopilación de estadísticas, etc.

In [ ]:
Datos_Originales_Sin_Datos_Faltantes = np.vstack((data['age'], datos_categoricos['workclass'], data['fnlwgt'], datos_categoricos['education'], data['education-num'],
                                    datos_categoricos['marital-status'], datos_categoricos['occupation'], datos_categoricos['relationship'], datos_categoricos['race'], 
                                    datos_categoricos['gender'], data['capital-gain'], data['capital-loss'], data['hours-per-week'],
                                    datos_categoricos['native-country'], data['income'])).T # Impotante aplicar la transpuesta

Datos_Originales_Sin_Datos_Faltantes = pd.DataFrame(Datos_Originales_Sin_Datos_Faltantes, columns = columnas)
display(Datos_Originales_Sin_Datos_Faltantes)

In [ ]:
Datos_Originales_Sin_Datos_Faltantes.info()

Antes de relacionar la característica numérica, se debe convertir los datos tipo *object* a *int* o *float*

In [ ]:
Datos_Originales_Sin_Datos_Faltantes['age'] = Datos_Originales_Sin_Datos_Faltantes['age'].astype('int64')
Datos_Originales_Sin_Datos_Faltantes['age']

Vamos a realizar un grupo de datos categorícos con datos numéricos

In [ ]:
datos_categoricos_agrupados = Datos_Originales_Sin_Datos_Faltantes.groupby(['workclass', 'native-country'])

Los datos categóricos agrupados de las características 'workclass' y 'native-country' se relacionarán con una característica cuantitativo llamado 'age'

In [ ]:
datos_agrupados_con_caracteristica_numerica = datos_categoricos_agrupados['age']

Vamos a agregar un estadístico como la *media* para relacionar los datos categóricos seleccionados con los datos numéricos de la edad

In [ ]:
datos_agrupados_con_caracteristica_numerica.agg('mean')

Como se puede apreciar, esta estrategía permite relacionar características categóricas con características cuantitativas, mediante un estadístico de interés. También podemos agregar otros estadísticos:

In [ ]:
datos_agrupados_con_caracteristica_numerica.agg(['median', 'std'])

Podemos agregar una función con varios estadísticos y métricas de interés

In [ ]:
funciones = ['count', 'mean', 'max']
Resultados = datos_agrupados_con_caracteristica_numerica.agg(funciones)
Resultados

Para observar todos los resultados, utilizamos las siguientes líneas de código

In [ ]:
pd.set_option('display.max_rows', 232) # Se colocan el numéro todas de filas que se obtiene de la variable **Resultados** 
display(Resultados)

Analicemos que sucede si relacionamos dos características cuantitativas con dos características categóricas

In [ ]:
Datos_Originales_Sin_Datos_Faltantes['age'] = Datos_Originales_Sin_Datos_Faltantes['age'].astype('int64')
Datos_Originales_Sin_Datos_Faltantes['hours-per-week'] = Datos_Originales_Sin_Datos_Faltantes['hours-per-week'].astype('int64')
datos_categoricos_agrupados = Datos_Originales_Sin_Datos_Faltantes.groupby(['workclass', 'native-country'])
datos_agrupados_con_caracteristica_numerica = datos_categoricos_agrupados[['age','hours-per-week']]

In [ ]:
datos_agrupados_con_caracteristica_numerica.agg({'age' : ['min', 'max', 'mean', 'std'], 'hours-per-week' : 'mean'})